## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

https://www.informatik.uni-leipzig.de/~saeedi/NCVoters_Readme.txt

```
5 party:
--------- 
5 sources. 
Each source 1000,000 entities.
There is one file per source, so totally 5 files
****************************************************
****************************************************
****************************************************
Fields:
---------- 
recId: entites with the same recId refer to the same entity.
givenname: 
surname: 
post code: 
suburb:
```

In [6]:
import glob
import csv
from tqdm.auto import tqdm

current_row_id = 0
row_dict = {}
rows_total = 5000000
cluster_attr = 'recid'

with tqdm(total=rows_total) as pbar:
    for filename in glob.glob(f'{home_dir}/Downloads/5Party-ocp20/*.csv'):
        with open(filename) as f:
            for row in csv.DictReader(f):
                row['id'] = current_row_id
                row[cluster_attr] = int(row[cluster_attr])  # convert cluster_attr to int
                row_dict[current_row_id] = row
                current_row_id += 1
                pbar.update(1)

In [7]:
row_dict[0]

{'recid': 7852009,
 'givenname': 'kadelyn',
 'surname': 'gragnani',
 'suburb': 'waxhaw',
 'postcode': '28|73',
 'id': 0}

## Preprocess

In [8]:
attr_list = ['givenname', 'surname', 'suburb', 'postcode']

In [9]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()[:30]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'givenname': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'surname': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'suburb': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'postcode': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

10:41:10 INFO:For attr='givenname', computing actual alphabet and max_str_len
10:41:14 INFO:For attr='givenname', using actual_max_str_len=16
10:41:14 INFO:For attr='surname', computing actual alphabet and max_str_len
10:41:18 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
10:41:18 INFO:For attr='surname', using actual_max_str_len=22
10:41:18 INFO:For attr='suburb', computing actual alphabet and max_str_len
10:41:22 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
10:41:22 INFO:For attr='suburb', using actual_max_str_len=22
10:41:22 INFO:For attr='postcode', computing actual alphabet and max_str_len
10:41:26 INFO:actual_max_str_len=9 must be pair to enable NN pooling. Updating to 10
10:41:26 INFO:For attr='postcode', using actual_max_str_len=10


{'givenname': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None),
 'surname': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=22, vocab=None),
 'suburb': NumericalizeInfo(field_type=<FieldType.STRI

In [14]:
from entity_embed import DeduplicationDataModule

train_cluster_len = 1000
valid_cluster_len = 1000
test_cluster_len = 200_000
datamodule = DeduplicationDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_numericalizer=row_numericalizer,
    batch_size=50,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=test_cluster_len,
    only_plural_clusters=True,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import EntityEmbed

ann_k = 10
model = EntityEmbed(
    datamodule,
    ann_k=ann_k,
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.5',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'voters'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

10:41:26 INFO:GPU available: True, used: True
10:41:26 INFO:TPU available: None, using: 0 TPU cores
10:41:26 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

10:41:39 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
10:41:42 INFO:Train pair count: 6679
10:41:42 INFO:Valid pair count: 6645
10:41:42 INFO:Test pair count: 1331364
10:41:45 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 2.5 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params


1

In [18]:
model.blocker_net.get_signature_weights()

{'givenname': 0.23119793832302094,
 'surname': 0.28396865725517273,
 'suburb': 0.2415730357170105,
 'postcode': 0.2432674765586853}

In [19]:
import gc
del trainer
gc.collect()

300

## Testing manually 

In [20]:
datamodule.setup(stage='test')

10:42:47 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
10:42:49 INFO:Train pair count: 6679
10:42:49 INFO:Valid pair count: 6645
10:42:49 INFO:Test pair count: 1331364


In [21]:
test_row_dict = datamodule.test_row_dict
test_vector_dict = model.predict(
    row_dict=test_row_dict,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [23]:
import gc
del model
del datamodule
gc.collect()

3149

In [24]:
assert len(test_vector_dict) == len(test_row_dict)

In [25]:
%%time

from entity_embed import ANNEntityIndex

ann_index = ANNEntityIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(test_vector_dict)
ann_index.build()

CPU times: user 11min 15s, sys: 2.41 s, total: 11min 17s
Wall time: 1min 20s


In [26]:
%%time

sim_threshold = 0.5
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold
)

CPU times: user 16min 36s, sys: 714 ms, total: 16min 37s
Wall time: 1min 49s


In [27]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

6.172743470459727

In [28]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.2698024443060213, 0.9997476272454415)

In [29]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

3602315

In [30]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

336

In [31]:
cos_similarity = lambda a, b: np.dot(a, b)

In [32]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_vector_dict[id_left], test_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.84432256,
 {'recid': 7839955,
  'givenname': 'danvs',
  'surname': 'johnson',
  'suburb': 'concord',
  'postcode': '28026',
  'id': 34454},
 {'recid': 7839955,
  'givenname': 'danuj',
  'surname': 'johhson',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 2034465})

(0.8184679,
 {'recid': 5584048,
  'givenname': 'angel',
  'surname': 'jones',
  'suburb': 'roanocke rapids',
  'postcode': '2787o',
  'id': 1042238},
 {'recid': 5584048,
  'givenname': 'ankel',
  'surname': 'jories',
  'suburb': 'roanoke rapids',
  'postcode': '27870',
  'id': 4042346})

(0.7029248,
 {'recid': 584874,
  'givenname': 'james',
  'surname': 'mcrae',
  'suburb': 'laurinburg',
  'postcode': '28352',
  'id': 853639},
 {'recid': 584874,
  'givenname': 'jamws',
  'surname': 'iiicrae',
  'suburb': 'laurinburg',
  'postcode': '28352',
  'id': 1016987})

(0.737379,
 {'recid': 2955051,
  'givenname': 'bret',
  'surname': 'dorr',
  'suburb': 'raleigh',
  'postcode': '276o3',
  'id': 68906},
 {'recid': 2955051,
  'givenname': 'prett',
  'surname': 'dorr',
  'suburb': 'raleigh',
  'postcode': '27609',
  'id': 2068956})

(0.9547178,
 {'recid': 6389260,
  'givenname': 'alexandriaq',
  'surname': 'mason',
  'suburb': 'ckannapolis',
  'postcode': '28081',
  'id': 1048684},
 {'recid': 6389260,
  'givenname': 'alexandria',
  'surname': 'mason',
  'suburb': 'kannapolis',
  'postcode': '28081',
  'id': 2934077})

(0.8650046,
 {'recid': 704391,
  'givenname': 'karol',
  'surname': 'mabe',
  'suburb': 'sandy ridge',
  'postcode': '27046',
  'id': 3499875},
 {'recid': 704391,
  'givenname': 'karok',
  'surname': 'mabe',
  'suburb': 'sandy ridge',
  'postcode': '27646',
  'id': 4019817})

(0.69735175,
 {'recid': 6217514,
  'givenname': 'maria',
  'surname': 'argis',
  'suburb': 'fayetteville',
  'postcode': '28363',
  'id': 35934},
 {'recid': 6217514,
  'givenname': 'mana',
  'surname': 'hargis',
  'suburb': 'fayetteville',
  'postcode': '29303',
  'id': 2035985})

(0.74558794,
 {'recid': 100998,
  'givenname': 'james',
  'surname': 'aynie',
  'suburb': 'winston salem',
  'postcode': '27101',
  'id': 2035950},
 {'recid': 100998,
  'givenname': 'sames',
  'surname': 'haynie',
  'suburb': 'winston salem',
  'postcode': '271o7',
  'id': 3036156})

(0.7030163,
 {'recid': 943532,
  'givenname': 'tere5a',
  'surname': 'iiiurray',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 1037384},
 {'recid': 943532,
  'givenname': 'teresa',
  'surname': 'murray',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 2961434})

(0.78150564,
 {'recid': 1844725,
  'givenname': 'papl',
  'surname': 'barne',
  'suburb': 'gastonia',
  'postcode': '28056',
  'id': 69108},
 {'recid': 1844725,
  'givenname': 'paul',
  'surname': 'pame',
  'suburb': 'gastonia',
  'postcode': '280s6',
  'id': 3069250})